這裡開始已經假設你已經看過前面的所有基礎文件說明，因此多數註解會拿掉以維護版面乾淨

ResNet的出現，讓模型沒有不深的理由，透過identity mapping讓output跟下兩個layer的output結合之後再做非線性的轉換，最重要的是，它不會造成梯度消失或爆炸，而且收斂效果非常的好。

在下已有翻譯ResNet論文，也可以參閱[相關文件](https://hackmd.io/@shaoeChen/SyjI6W2zB/https%3A%2F%2Fhackmd.io%2F%40shaoeChen%2FSy_e1mCEU)

首先載入相關需求套件

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
tf.__version__

'2.1.0'

指定硬體資源

In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')

資料集的部份是使用ImageNet訓練，不過這部份在下就只提供[資料集連結](http://www.image-net.org/)，不然硬train一發怕時間太久。

從論文中我們知道：
* Figure 5.說明，ResNet的殘差塊是由三個conv所構成
* ResNet的殘差塊是將上個殘差塊的input與第三個conv的output相加之後再做relu
* Figure 3.說明，每個conv stage的開始如果有維度上的調整，就需要利用B方案來做維度調整，以將上一個stage的output做shortcut connection
* Table 1.說明，ResNet50的架構

範例參考[keras application resnet](https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py)

我們需要建立一個殘差塊的函數來處理這一塊，殘差塊由三個卷積所構成，而且過程中的維度是不變的

In [4]:
def identity_mapping(pre_input, kernel_size, filters, stage, block):
    """identity block
    
    parameters:
        pre_input：上一個stage的output
        kernel_size: 跟著論文走的話，第一、三個conv是1x1，第二個是3x3
        filters：filter的數量，記得殘差塊是由三個conv所組成，所以這邊我們會給三個conv的filter數值
        stage: 第幾個identity stage
        block: 第幾個block
    """
    
    # 殘差塊是由三個卷積所組成，因此我們會定義三個卷積的filter數量
    filter1, filter2, filter3 = filters
    
    # 定義layer name，這對你觀察模型非常有幫助
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 我們會將上一個殘差塊的輸出，也就是這一個殘差塊的輸入與這一個殘差塊的輸出相加

    # 第一個conv, filter size=1x1
    x = tf.keras.layers.Conv2D(filters=filter1, 
                               kernel_size=1, 
                               strides=1, 
                               name=conv_name_base + '2a')(pre_input)
    x = tf.keras.layers.BatchNormalization(name=bn_name_base + '2a')(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # 第二個conv, filter size=kernel_size
    x = tf.keras.layers.Conv2D(filters=filter2, 
                               kernel_size=kernel_size, 
                               strides=1, 
                               padding='same', 
                               name=conv_name_base + '2b')(x)
    x = tf.keras.layers.BatchNormalization(name=bn_name_base + '2b')(x)
    x = tf.keras.layers.Activation('relu')(x)    
    
    # 第三個conv, filter size=1x1
    x = tf.keras.layers.Conv2D(filters=filter3, 
                               kernel_size=1,
                               strides=1, 
                               name=conv_name_base + '2c')(x)
    x = tf.keras.layers.BatchNormalization(name=bn_name_base + '2c')(x)
    # 要注意，這邊並沒有先做非線性計算
    
    # 計算shortcut 
    # 上一個殘差塊的input會跟第三個conv的output相加之後再relu
    # 因為維度上並沒有改變，因此不需要做維度調整
    # 想像一下，conv2_x的第一個block的output的channel是256，第二個block的channel也是256
    # 過程中又是利用same padding，因此維度是不變的，可以直接相加
    x = tf.keras.layers.Add()([x, pre_input])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x
    

接著是標準的conv layer block，論文中有提到，每個stage的開頭都是stride=2來執行downsampling，當有維度不同的時候，就利用projection shortcut(plan B)來處理

In [21]:
def conv_block(pre_input, kernel_size, filters,  stage, block, strides=2):
    """維度變更調整
    
    parameters:
        pre_input：上一個stage的output
        kernel_size: 跟著論文走的話，第一、三個conv是1x1，第二個是3x3
        filters：filter的數量，記得殘差塊是由三個conv所組成，所以這邊我們會給三個conv的filter數值
        stage: 第幾個identity stage
        block: 第幾個block
        strides: 步幅
    """
    
    # 定義名稱
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # 取得三個卷積層的filter數
    filter1, filter2, filter3 = filters
    
    # 與identity mapping是三個conv layer所組成
    # 第一個conv layer
    # 依著論文，第一個conv的stride會是2，讓dimension減半
    x = tf.keras.layers.Conv2D(filters=filter1, 
                               kernel_size=1, 
                               strides=strides, 
                               name=conv_name_base + '2a',
                               kernel_initializer=tf.keras.initializers.glorot_uniform(seed=10)
                               )(pre_input)
    x = tf.keras.layers.BatchNormalization(name=bn_name_base + '2a')(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # 第二個conv layer
    x = tf.keras.layers.Conv2D(filters=filter2,
                               kernel_size=kernel_size,
                               strides=1,
                               padding='same',
                               name=conv_name_base + '2b',
                               kernel_initializer=tf.keras.initializers.glorot_uniform(seed=10)
                               )(x)
    x = tf.keras.layers.BatchNormalization(name=bn_name_base + '2b')(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # 第三個conv layer
    x = tf.keras.layers.Conv2D(filters=filter3,
                               kernel_size=1,
                               strides=1,
                               name=conv_name_base + '2c',
                               kernel_initializer=tf.keras.initializers.glorot_uniform(seed=10)
                              )(x)
    x = tf.keras.layers.BatchNormalization(name=bn_name_base + '2c')(x)
    
    # shortcut
    # 讓input的channel先利用1x1 conv調整為等同第三個conv layer的channel
    # 然後與第一個conv layer一樣的strides，這樣子dimension才會與第三個conv layer的output一致
    # 這是為了讓兩個矩陣可以相加，神經網路最重要的其中一點就是維度的控制確認
    shortcut = tf.keras.layers.Conv2D(filters=filter3, 
                                      kernel_size=1,
                                      strides=strides, 
                                      name=conv_name_base + '1',
                                      kernel_initializer=tf.keras.initializers.glorot_uniform(seed=10)
                                     )(pre_input)
    shortcut = tf.keras.layers.BatchNormalization(name=bn_name_base + '1')(shortcut)
    
    # 最後，將兩個output相加之後再執行relu
    x = tf.keras.layers.Add()([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x
    

現在，我們已經建立好論文中的Figure 3中的虛線與實線連結的block，要注意的是Figure 3所示是兩個conv layer一組，但後續調整為三個一組，有著較好的效能。

現在來建置模型吧

In [22]:
def resnet50():
    """building resnet50
    
    從論文的Table 1可以清楚看的出來，整個resnet有五個stage，每個stage有各自n個block
    """
    x_input = tf.keras.layers.Input((224, 224, 3))
    x = tf.keras.layers.ZeroPadding2D((3, 3), name='zero_padding')(x_input)
    
    # stage1
    x = tf.keras.layers.Conv2D(64, 
                               kernel_size=7, 
                               strides=2, 
                               name='conv1', 
                               kernel_initializer=tf.keras.initializers.glorot_uniform(seed=10)
                              )(x)
    x = tf.keras.layers.BatchNormalization(name='bn1')(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # stage2
    # 看Table 1，maxpooling應該是stage2...所以我就把它列在這邊
    x = tf.keras.layers.MaxPooling2D(pool_size=3, strides=2)(x)
    x = conv_block(x, kernel_size=3, filters=[64, 64, 256], stage=2, block='a', strides=1)
    x = identity_mapping(x, kernel_size=3, filters=[64, 64, 256], stage=2, block='b') 
    x = identity_mapping(x, kernel_size=3, filters=[64, 64, 256], stage=2, block='c')
    
#     # stage3
    x = conv_block(x, kernel_size=3, filters=[128, 128, 512], stage=3, block='a', strides=2)
    x = identity_mapping(x, kernel_size=3, filters=[128, 128, 512], stage=3, block='b') 
    x = identity_mapping(x, kernel_size=3, filters=[128, 128, 512], stage=3, block='c')
    x = identity_mapping(x, kernel_size=3, filters=[128, 128, 512], stage=3, block='d')
    
    # stage4
    x = conv_block(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='a', strides=2)
    x = identity_mapping(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='b') 
    x = identity_mapping(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='c') 
    x = identity_mapping(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='d') 
    x = identity_mapping(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='e') 
    x = identity_mapping(x, kernel_size=3, filters=[256, 256, 1024], stage=4, block='f') 
    
#     # stage5
    x = conv_block(x, kernel_size=3, filters=[512, 512, 2048], stage=5, block='a', strides=2)
    x = identity_mapping(x, kernel_size=3, filters=[512, 512, 2048], stage=5, block='b') 
    x = identity_mapping(x, kernel_size=3, filters=[512, 512, 2048], stage=5, block='c') 
    
    # average pooling
    x = tf.keras.layers.AveragePooling2D(pool_size=(2, 2), name='avg_pooling')(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(units=1000, 
                              activation='softmax', 
                              name='softmax', 
                              kernel_initializer=tf.keras.initializers.glorot_uniform(seed=10)
                             )(x)
    
    
    # build model
    model = tf.keras.models.Model(inputs=x_input, outputs=x, name='ResNet50')
    
    return model

現在來生成一個模型，然後觀察一下吧。

In [19]:
model_res50 = resnet50()

In [20]:
model_res50.summary()


Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding (ZeroPadding2D)    (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding[0][0]               
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

可以注意到，假設input=224x244x3，在經過第二個stage之後的output是55，這是第三個stage的input，但經過第三個stage的第一個block之後其維度剩28，這種情況下你要將兩個不同維度的矩陣相加是不可能的，也因此在conv_block的shortcut中我們設置這個shortcut的filters為filter3的channel，然後讓它的stride與第一個block一樣，這樣子就可以產生與filter3相同維度的輸出，利用這個方式轉換維度，讓上一個stage的output可以跟這一個stage的output相加，達成效果。

編譯模型

In [17]:
model_res50.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy']
)

ResNet深深影響後續的模型，它可達到一千多層，但依然可以訓練，Inception_v4也加入這個概念，我們需要更深。